# Getting started with WFSim

Hello friend.
Welcome to the basic tutorial on how to simulate waveforms with the latest wfsim version in strax.
Here we'll just demonstrate the basic functionality. For more in depth analysis stuff, checkout the straxen tutorials for more detailed thing.

In [1]:
import numpy as np
import strax
import straxen
import wfsim

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from multihist import Histdd, Hist1d
from scipy import stats

import json

*** Detector definition message ***
You are currently using the default XENON10 template detector.



## Different simulators

Because we'd like to make simulations we have multiple different ways of making data. They are:
* `RawRecordsFromFaxNT(1T)`
* `RawRecordsFromFaxEpix`
* `RawRecordsFromFaxOptical`
* `RawRecordsFromFaxNveto`

The main difference is the way you specify instructions.  The NT(1T) simulator depends on either a csv file or a defaul/user specified function to generate instructions. Epix takes input from G4 processed with  epix. Optical takes optical input from G4 for photon channels and timings. And Nveto is a small modification to optical to provide nveto data

## Setting everything up

First we need to define the right context. For this we made two contexts to help you out. One for 1T and one for nT. By default in your context RawRecordsFromFaxNT will be registered. If you  want another one be sure to register it

For 1T do:

In [2]:
st = straxen.contexts.xenon1t_simulation()

Or nT do:

In [3]:
st = straxen.contexts.xenonnt_simulation(cmt_run_id_sim='00001')

Now we need to define a run id. What you give it doesn't really matter, since strax will look for data and make new if it doesn't find anything. And this is what you want.
Strax will use the run id to get the electron lifetime and pmt gains from a database, and returns placeholders if the run doesn't exist.

In [4]:
run_id = '1'

## Defining instructions


For the instructions there are multiple different ways to do it. The simulator has this option called "fax_file". If it has a value (None by default) the simulator will either read it as a csv or root file. If not it will use some predefined functions to make your events. The number of event you'll simulate based on the product of the config values "nchunk","event_rate" and "chunk_size". Which you can set as follows:

In [5]:
st.set_config(dict(nchunk=1, event_rate=5, chunk_size=10,))

Strax groups data together in chunks based on time (for low level data). nchunk is the number of chunks you want to simulate
event_rate is the number of events per seconds, so this effects the amount of spacing between events. Finally chunk_size is the length in seconds of your chunk
For the DAQ this is 5 seconds. For simulations you can do whatever you want. It is important to note that Strax will write out data per chunk
So when your chunks are small you'll, among other things, call Strax' IO functions a lot, giving a substantial overhead. On the other hand, to large chunks will hog all your memory and your kernel might crash.
Based on my experimentation setting chunk_size to 500 gives best performance

These are ways you can give instructions
### Random
This is the default where simulator will generate some random instructions for you.
### Custom
For this you will need to overwrite the instruction generator function
### CSV
You can provide a csv file with the instruction (Like the output of nSort)
### Geant4
For this you'll  need to  use epix to do some clustering for you.

### Random
I guess this is pretty self explanatory. The simulator has this function called "rand_instructions" which will make something up for you.



In [6]:
wfsim.strax_interface.rand_instructions??

Signature: wfsim.strax_interface.rand_instructions(c)
Source:   
def rand_instructions(c):
    """Random instruction generator function. This will be called by wfsim if you do not specify 
    specific instructions.
    :params c: wfsim configuration dict"""
    n = c['nevents'] = c['event_rate'] * c['chunk_size'] * c['nchunk']
    c['total_time'] = c['chunk_size'] * c['nchunk']

    instructions = np.zeros(2 * n, dtype=instruction_dtype)
    uniform_times = c['total_time'] * (np.arange(n) + 1.) / n
    instructions['time'] = np.repeat(uniform_times, 2) * int(1e9)
    instructions['event_number'] = np.digitize(instructions['time'],
                                               1e9 * np.arange(c['nchunk']) * c['chunk_size']) - 1
    instructions['type'] = np.tile([1, 2], n)
    instructions['recoil'] = np.repeat(7, n * 2)  # Use nest ids for  ER

    r = np.sqrt(np.random.uniform(0, c['tpc_radius']**2, n))
    t = np.random.uniform(-np.pi, np.pi, n)
    instructions['x'] = np.repeat(r 

### Custom
This is some more fun. To do this we'll write a new function which returns a structured numpy array with the correct dtype.

In [7]:
wfsim.instruction_dtype

[(('Waveform simulator event number.', 'event_number'), numpy.int32),
 (('Quanta type (S1 photons or S2 electrons)', 'type'), numpy.int8),
 (('Time of the interaction [ns]', 'time'), numpy.int64),
 (('X position of the cluster[cm]', 'x'), numpy.float32),
 (('Y position of the cluster[cm]', 'y'), numpy.float32),
 (('Z position of the cluster[cm]', 'z'), numpy.float32),
 (('Number of quanta', 'amp'), numpy.int32),
 (('Recoil type of interaction.', 'recoil'), numpy.int8),
 (('Energy deposit of interaction', 'e_dep'), numpy.float32),
 (('Eventid like in geant4 output rootfile', 'g4id'), numpy.int32),
 (('Volume id giving the detector subvolume', 'vol_id'), numpy.int32),
 (('Local field [ V / cm ]', 'local_field'), numpy.float64),
 (('Number of excitons', 'n_excitons'), numpy.int32)]

Event number is just a lable which peaks are together. type is either 1(S1) or 2 (S2). In the truth higher numbers are also used to refer to different types of afterpulses. time,x,y,z are the time and positions of the signal. Amp is the number of photons or electrons generated, and recoil can be used for different types of recoil (but only Electronic recoil is supported). Recoil is a number according to the NEST convention which will be used to indicate which interaction. For the key look here: https://github.com/XENONnT/WFSim/blob/2c614b0f7b0d7c7adc516f6188e281857e8d7e22/wfsim/core.py#L22

Now, lets say we want some krypton peaks. For this we'll need to change the default instruction function to include this double decay and use Nestpy to convert energy deposits into a number of photons and electrons.

In this case I'll use 1 "event" per full decay, that where all the 4's are coming from.

In [8]:
def super_awesome_custom_instruction(c):
    import nestpy
    half_life = 156.94e-9 #Kr intermediate state half-life in ns
    decay_energies = [32.2,9.4] # Decay energies in kev
    
    n = c['nevents'] = c['event_rate'] * c['chunk_size'] * c['nchunk']
    c['total_time'] = c['chunk_size'] * c['nchunk']

    instructions = np.zeros(4 * n, dtype=wfsim.instruction_dtype)
    instructions['event_number'] = np.digitize(instructions['time'],
         1e9 * np.arange(c['nchunk']) * c['chunk_size']) - 1
    
    instructions['type'] = np.tile([1, 2], 2 * n)
    instructions['recoil'] = [7 for i in range(4 * n)]
    
    r = np.sqrt(np.random.uniform(0, 2500, n))
    t = np.random.uniform(-np.pi, np.pi, n)
    instructions['x'] = np.repeat(r * np.cos(t), 4)
    instructions['y'] = np.repeat(r * np.sin(t), 4)
    instructions['z'] = np.repeat(np.random.uniform(-100, 0, n), 4)
    
    #To get the correct times we'll need to include the 156.94 ns half life of the intermediate state.

    uniform_times = c['total_time'] * (np.arange(n) + 0.5) / n
    delayed_times = uniform_times + np.random.exponential(half_life/np.log(2),len(uniform_times))
    instructions['time'] = np.repeat(list(zip(uniform_times,delayed_times)),2) * 1e9


    # Here we'll define our XENON-like detector
    nc = nestpy.NESTcalc(nestpy.VDetector())
    A = 131.293
    Z = 54.
    density = 2.862  # g/cm^3   #SR1 Value
    drift_field = 82  # V/cm    #SR1 Value
    interaction = nestpy.INTERACTION_TYPE(7)
    
    energy = np.tile(decay_energies,n)
    quanta = []
    exciton = []
    for en in energy:
        y = nc.GetYields(interaction,
                         en,
                         density,
                         drift_field,
                         A,
                         Z,
                         (1, 1))
        q = nc.GetQuanta(y, density)
        quanta.append(q.photons)
        quanta.append(q.electrons)
        exciton.append(q.excitons)
        exciton.append(0)
        
    instructions['amp'] = quanta
    instructions['local_field'] = drift_field
    instructions['n_excitons'] = exciton

    return instructions

Now here comes the magic line:

In [9]:
wfsim.strax_interface.rand_instructions = super_awesome_custom_instruction

This changes the default rand_instruction function in our own super awesome function. So when the simulator will call rand_instruction the code from super_awesome_custom_instruction will be executed

### CSV
The format for csv files is the same as the instructions dtype. So on every line specify event_number,type,time ,x,y,z, amp and recoil in that order.
Then tell the simulator it exists:


In [10]:
st.set_config(dict(fax_file='instructions.csv'))

Ofcourse if you do not have this file it will not work

### Geant4
For starters you'll need to register the Epix plugin. This requires an epix configuration (it's on private_nt_aux_files), and if you want a start and stop event.

In [11]:
st.register(wfsim.strax_interface.RawRecordsFromMcChain)
st.set_config(dict(fax_file    = 'path_to_g4_file',
                   epix_config = 'path_to_config'))

### Optical

Similar to above, register correct plugin and file:

In [12]:
st.register(wfsim.strax_interface.RawRecordsFromMcChain)
st.set_config(dict(fax_file = 'path_to_g4_file',))

### nVeto

Because the nveto is a different system it has a slightly different configuration. This is not entirely finished. Once it is you can run like so:

In [13]:
st.register(wfsim.strax_interface.RawRecordsFromFaxnVeto)
st.set_config(dict(fax_config_nveto = 'path_to_nveto_config_file',
                  fax_file          = 'path_to_g4_file',))

## Configuration customization
The simulator using a larger large amount of configuration settings to do it's magic. Some of them are best left along, like pmt_circuit_load_resistor. Others on the other hand are things you might want change a bit to see how the data will change. Unfortunately currently the full list is spread out over two different places. One is the fax config json which is on github. The other is the option list in strax. Besides those things like pattern maps are hardcoded in load_resource.py. These files are overrideable with the option "fax_config_override"

The strax config is viewable like this and can be changed by st.set_config(dict(option you want=value you want))

In [14]:
st.show_config('raw_records')

,option,default,current,applies_to,help
0,detector,XENONnT,XENONnT,"(raw_records, raw_records_he, raw_records_aqmo...",
1,event_rate,1000,5,"(raw_records, raw_records_he, raw_records_aqmo...",Average number of events per second
2,chunk_size,100,10,"(raw_records, raw_records_he, raw_records_aqmo...",Duration of each chunk in seconds
3,nchunk,10,1,"(raw_records, raw_records_he, raw_records_aqmo...",Number of chunks to simulate
4,fax_file,None,path_to_g4_file,"(raw_records, raw_records_he, raw_records_aqmo...",Directory with fax instructions
5,fax_config,fax_config_nt_design.json,fax_config_nt_design.json,"(raw_records, raw_records_he, raw_records_aqmo...",
6,fax_config_override,None,<OMITTED>,"(raw_records, raw_records_he, raw_records_aqmo...",Dictionary with configuration option overrides
7,fax_config_override_from_cmt,None,"{'drift_time_gate': ('cmt_run_id', '00001', 'e...","(raw_records, raw_records_he, raw_records_aqmo...",Dictionary of fax parameter names (key) mapped...
8,gain_model_mc,"(to_pe_per_run, to_pe_nt.npy)","(cmt_run_id, 00001, to_pe_model, v4, True)","(raw_records, raw_records_he, raw_records_aqmo...","PMT gain model. Specify as (model_type, model_..."
9,channel_map,<OMITTED>,"(tpc, he, aqmon, aqmon_nv, tpc_blank, mv, aux_...","(raw_records, raw_records_he, raw_records_aqmo...","immutabledict mapping subdetector to (min, max..."


With the following block you can visualized which fax config you are using. Remeber that the fax config files are stored in **private_nt_aux_files** github repository, in the **sim_files** folder.

In [15]:
st.config['fax_config']

'fax_config_nt_design.json'

The config from github can be loaded as:

In [16]:
straxen.get_resource('fax_config_nt_design.json',fmt='json').keys()

dict_keys(['s1_model_type', 's2_luminescence_model', 'enable_gas_gap_warping', 'enable_pmt_afterpulses', 'enable_electron_afterpulses', 'enable_noise', 'field_distortion_on', 'enable_field_dependencies', 'url_base', 'photon_area_distribution', 's1_pattern_map', 's2_pattern_map', 'photon_ap_cdfs', 's2_luminescence', 'gas_gap_map', 'temperature', 'pressure', 'lxe_dielectric_constant', 'tpc_length', 'tpc_radius', 'anode_wire_radius', 'anode_field_domination_distance', 'elr_gas_gap_length', 'gate_to_anode_distance', 'drift_field', 'anode_voltage', 'diffusion_constant_longitudinal', 'diffusion_constant_transverse', 'drift_time_gate', 'drift_velocity_liquid', 'singlet_fraction_gas', 'singlet_lifetime_gas', 'singlet_lifetime_liquid', 'triplet_lifetime_gas', 'triplet_lifetime_liquid', 's1_ER_alpha_singlet_fraction', 's1_ER_primary_singlet_fraction', 's1_ER_recombination_fraction', 's1_ER_recombination_time', 's1_ER_secondary_singlet_fraction', 's1_NR_singlet_fraction', 'maximum_recombination_t

Changing things in this guy goes slightly different. In the strax option list there is the option called "fax_config_override". This takes a dict which will be used to override any values in the json config.
So changing the 's2_secondary_sc_gain' is done as:

In [17]:
st.set_config(dict(fax_config_override = dict(s2_secondary_sc_gain=23)))

## Things you might want to change

There are 4 main things you might be interested in changing. They are:
 * Noise (True/False)
 * PMTAfterpulses (True/False)
 * Electron Afterpulses (True/False)
 * S2 luminescence (Simple or garfield)

They are controled by config settings you can change as so:


In [18]:
st.set_config(dict(fax_config_override=dict(
            s2_luminescence_model='simple',
            enable_noise=False,
            enable_pmt_afterpulses=False,
            enable_electron_afterpulses=False,)))
    

## What actually happens?



What happens behind the scenes is that the instructions are first grouped together in chunks. Then we loop over the instructions and the full chunk is returned before starting with the next one.

We use a S1 and S2 class to calculate the arrival times of the photons and the channels which have been hit. Then we'll hand them over to the Pulse class to calculate the currents in the channels. Finally the currents go to a RawData class where we fake the digitizer response.

### S1

For S1s we start with calculating the light yield based on the position of the interaction, and draw the number of photons seen from a Poisson distribution.

Second we calculate the arrival times of the photons. This is based on the scintillation of the xenon atoms. It is dependend on the recombination time, the singlet and triplet fractions.

Finally the channels are calculated. Based on the pattern map we use a interpolation map to get a probability distribution for channels to be hit for a S1 signal based on the position of the interaction, and then we draw from this distribution for every photon.

### S2

S2s are slightly more complicated. First we need to drift the electrons up, and while doing so we'll lose some of them.
To get the photon timings, we first need to get the arrival times of the electrons at the gas interface based on a diffusion model. Then we can calculate the photon timings based on a luminescence model for every individual electron. And for the channels we do the same trick with the interpolating map.


### Pulse

When we have our lists of channels and timing we can generate actual pulses. First we add a pmt transition time. Then we loop over all channels, calculate the double pe emission probabilities, and add a current in the pmt channel based on the arrival time. This is all stored in a big dictionary. Afterwards this is passed to our fake digitizers which then returns you with your very own pretty data


### Getting down to bussiness


Now we have access to all the normal strax data types, and another one called 'truth' which holds the simulation instructions. Calling it follows the normal strax convention.

In [19]:
st = straxen.contexts.xenonnt_simulation(cmt_run_id_sim='00001')
st.set_config(dict(nchunk=1, event_rate=5, chunk_size=5, fax_file=None))

In [20]:
# Remove any previously simulated data, if such exists
# !rm -r strax_data

records = st.get_array(run_id,'records')
# peaks = st.get_array(run_id, ['peaks','peak_basics'])
# data = st.get_df(run_id, 'event_info')

truth = st.get_df(run_id, 'truth')

Loading records: |          | 0.00 % [00:00<?]

Loading truth: |          | 0.00 % [00:00<?]

Now it is time to make pretty plots and see if what we makes actually makes any sense

In [21]:
peak_basics = st.get_df(run_id, 'peak_basics')

Loading peak_basics: |          | 0.00 % [00:00<?]

In [22]:
peak_basics.head(10)

,time,endtime,center_time,area,n_channels,max_pmt,max_pmt_area,range_50p_area,range_90p_area,area_fraction_top,length,dt,rise_time,tight_coincidence,type
0,100000030,100000340,100000116,6.008117,2,487,3.396131,84.658058,139.222778,0.434743,31,10,72.558640,1,0
1,100085020,100090330,100086781,6050.895996,135,138,1673.483521,805.982056,2172.673828,0.933084,177,30,786.898987,34,2
2,300000050,300000360,300000139,9.503628,2,111,5.884011,71.074753,175.408966,0.619133,31,10,59.963516,1,0
3,300028280,300032720,300029916,11545.807617,181,32,2268.296875,636.287964,1483.845093,0.900317,148,30,584.498291,61,2
4,300035520,300037250,300036432,17.388079,7,134,3.215723,884.486389,1485.202148,1.000000,173,10,934.491943,2,2
5,500000040,500000570,500000244,19.048435,5,375,5.513971,220.591858,359.682007,0.000000,53,10,149.986282,2,2
6,500960650,500960920,500960710,6.818991,2,128,4.352914,43.098900,83.738449,0.638352,27,10,30.770397,2,1
7,700000050,700000420,700000145,18.850853,6,373,4.941594,102.237549,178.564667,0.105509,37,10,38.637653,4,1
8,700001130,700001520,700001257,4.998666,2,423,2.648085,145.234467,272.601288,0.000000,39,10,79.824738,1,0
9,700599860,700606020,700602521,110.203194,15,99,37.721416,3226.068115,5537.778809,0.930984,154,40,2111.939453,2,2


## Matching
To do matching with the truth the easiest way is to write a new strax plugin where you loop over peaks and get the truth arrays where the mean arrival time of the photons are within the time window of the peak
So that will look something like this:

In [23]:
class MatchedPeaks(strax.LoopPlugin):
    depends_on = ('peak_basics', 'truth')
    provides = 'matched_peaks'
    __version__ = '0.0.2'

    dtype = [('time', np.int),
             ('endtime', np.int),
             ('area', np.int),
             ('n_photon', np.int)]
    
    def compute(self, peaks, truth):
        result = np.zeros(len(truth), self.dtype)

        temp = strax.touching_windows(peaks, truth)
        idx_truth = []
        idx_peak = []
        for ix, (l, r) in enumerate(temp):
            res = result[ix]
            res['time'] = truth['time'][ix]
            res['endtime'] = strax.endtime(truth)[ix]
            res['n_photon'] = truth['n_photon'][ix]
    
            if r > l:
                i_peak = l+np.argmax(peaks['area'][l:r])
                res['area'] = peaks['area'][i_peak]
        
        return result

Of course this doesn't actually work. An electron afterpulse can be very spread out leading it to be interpreted as multiple peaks while coming from 1 instruction falling outside of the specified range. It would be very much appreciated if someone wants make a more sturdy selection criteria :)

For example, checkout out peak matching algorithm for WFSim: https://github.com/XENONnT/pema

In [24]:
st.register(MatchedPeaks)

__main__.MatchedPeaks

In [25]:
st.get_df(run_id, 'matched_peaks').head(10)

Loading matched_peaks: |          | 0.00 % [00:00<?]

,time,endtime,area,n_photon
0,100000026,100000603,6,219
1,100000337,100000841,6,70
2,100084342,100089998,6050,15593
3,100091453,100095505,0,392
4,100184771,100186098,0,58
5,300000026,300000831,9,176
6,300000248,300000759,9,71
7,300028152,300032297,11545,23189
8,300035169,300039343,17,545
9,300051579,300054633,0,59


# Externally call functions

The s1/s2 functions used for generating photon timings and channels are now doocumented and callable from the outside. This you can use if you want to do something like compare photon arrival timings between wfsims results and g4

In [26]:
wfsim.core.S1.get_n_photons?

Object `wfsim.core.S1.get_n_photons` not found.
